In [11]:
from __future__ import unicode_literals
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [12]:
#pd.options.display.max_columns = None
pd.options.display.max_colwidth = 1

In [13]:
train = pd.read_csv('../ItemInfo_trainfull.csv', encoding='utf-8', nrows=400)

In [17]:
train.head()

,itemID,categoryID_x,title_x,description_x,images_array_x,attrsJSON_x,price_x,locationID_x,metroID_x,lat_x,lon_x,itemID_2,isDuplicate,generationMethod,categoryID_y,title_y,description_y,images_array_y,attrsJSON_y,price_y,locationID_y,metroID_y,lat_y,lon_y,parentCategoryID_x,parentCategoryID_y,regionID_x,regionID_y,lendiff_imagearray,priceDifference,latlonDifference,fuzz_ratio,lev_dist,jaro_dist,jarow_dist,description_x_clean,description_y_clean,intersect_BOW,sym_diff_BOW,clusters,desc_sim,City_x,Neighborhood_x,State_x,Street_x,geocodeQuality_x,postalCode_x
0,1,81,Продам Камаз 6520,Продам Камаз 6520 20 тонн,"1064094, 5252822, 6645873, 6960145, 9230265","{""Вид техники"":""Грузовики""}",300000.0,648140,NaN,64.686946,30.815924,4112648,1,1,81,Продам Камаз 6520,Продам Камаз 6520 20 тонн,"1227519, 1374615, 7072137, 8671835","{""Вид техники"":""Грузовики""}",300000.0,648140,NaN,64.686946,30.815924,1,1,648070,648070,1.0,0.0,0.000000,1.0,0,1.000000,1.000000,"[продам, камаз, 6520, 20, тонн]","[продам, камаз, 6520, 20, тонн]","[6520, камаз, продам, тонн, 20]",[],0,1.000000,NaN,NaN,Northwestern Federal District,NaN,STATE,NaN
1,3,14,Yamaha r6,Весь в тюнинге.,"11919573, 14412228, 3204180, 6646877","{""Вид техники"":""Мотоциклы"", ""Вид мотоцикла"":""Спортивные""}",300000.0,639040,NaN,55.678037,37.256548,1991275,1,1,14,Yamaha R6,Весь в тюнинге. Возможен торг,"11068709, 13325040, 13783238, 206652, 9458537","{""Вид техники"":""Мотоциклы"", ""Вид мотоцикла"":""Спортивные""}",330000.0,639040,NaN,55.678037,37.256548,1,1,637680,637680,1.0,30000.0,0.000000,1.0,1,0.925926,0.955556,[тюнинге],"[тюнинге, возможен, торг]",[тюнинге],"[возможен, торг]",0,0.698324,Odintsovo,NaN,Moscow Oblast,Marshala Jhukova Street,STREET,143000
2,4,84,iPhone 3gs 8gb,"Телефон в хорошем состоянии, трещин и сколов нет, за все время менялся только аккумулятор(поэтому заряд держит хорошо), остальное все родное, в целом работает отлично! В комплекте кабель. Обмен не интересен. Лежит без дела, только продажа.","14384831, 6102021","{""Вид телефона"":""iPhone""}",3500.0,640650,NaN,56.239398,43.460458,1223296,0,1,84,iPhone 3gs 32gb,"В отличном состоянии, в комплекте зарядник и кабель. Без обмена.",5709245,"{""Вид телефона"":""iPhone""}",3500.0,640650,NaN,56.239398,43.460458,6,6,640310,640310,1.0,0.0,0.000000,0.9,2,0.931746,0.959048,"[телефон, хорошем, состоянии, трещин, сколов, менялся, аккумуляторпоэтому, заряд, держит, остальное, родное, целом, работает, отлично, комплекте, кабель, обмен, интересен, лежит, дела, продажа]","[отличном, состоянии, комплекте, зарядник, кабель, обмена]","[состоянии, комплекте, кабель]","[работает, держит, лежит, зарядник, родное, интересен, менялся, хорошем, обмен, целом, аккумуляторпоэтому, телефон, сколов, остальное, обмена, отлично, заряд, отличном, дела, продажа, трещин]",0,0.902371,Dzerzhinsk,NaN,Nizhny Novgorod Oblast,1 улица Кирова,POINT,606024
3,7,84,Xiaomi Mi4 3гб RAM + 16гб ROM белый,"Отличный подарок на новый год от ""китайской apple""\nНовый в упаковке. Коробку вместе вскроем)\nЭкран: 5 дюймов, 1080p, Sharp/JDI OGS (Возможно использование в перчатках)\nЦвет корпуса: белый\nМатериалы корпуса: пластик, металл, стекло\n\nПроцессор: Qualcomm Snapdragon 801 (8X74AC, 2.5Ггц)\nВидео: Adreno 330 (578 Мгц)\nОЗУ: 3Гб (LPDDR3 933Мгц)\nПамять: 16Гб (eMMC 5.0)\n\nКамера задняя: 13Мп (Sony IMX214, f1.8, 6 линз)\nКамера передняя: 8Мп (Sony IMX219, f1.8, 5 линз)\n\nWiFi: 802.11 a/b/g/n/ac 2.4/5Ггц\nBluetooth: 4.0\nНавигация: GPS/Glonass\nСенсоры: ИК-Порт, Датчик освещенности, Барометр, Компас, Датчик силы тяжести\n\nСимкарта: MicroSIM\n\nАккумулятор: 3080 мАч (быстрая зарядка при 9V/1.2A или 5V/2A)\n\nДополнительно: Поддержка Smart Cover Case\n\nШирина: 68,5 мм\nВысота: 139.2 мм\nТолщина: 8.9 мм\nВес: 149 г\nБез 4G",NaN,"{""Вид телефона"":""Другие марки""}",13500.0,662210,NaN,55.777170,37.586194,1058851,1,1,84,Смартфон Xiaomi Mi4 3гб RAM + 16гб ROM белый,"Отличный подарок на новый год от ""китайской apple""\nНовый в упаковке. Коробку вместе вскроем)\nЭкра

In [18]:
# Null values are numerical, impute with zero
train = train.fillna(value=0,axis=1)

In [19]:
nonnum_columns = [key for key in dict(train.dtypes) if dict(train.dtypes)[key] in ['object']]

In [ ]:
# Memory Error
# ohe = OneHotEncoder()

In [20]:
le = LabelEncoder()

In [23]:
for feature in nonnum_columns:
#     print(feature)
#     print(type(feature))
#     print(train[feature])
    train[feature] = le.fit_transform(train[feature])

In [24]:
train.head()

,itemID,categoryID_x,title_x,description_x,images_array_x,attrsJSON_x,price_x,locationID_x,metroID_x,lat_x,lon_x,itemID_2,isDuplicate,generationMethod,categoryID_y,title_y,description_y,images_array_y,attrsJSON_y,price_y,locationID_y,metroID_y,lat_y,lon_y,parentCategoryID_x,parentCategoryID_y,regionID_x,regionID_y,lendiff_imagearray,priceDifference,latlonDifference,fuzz_ratio,lev_dist,jaro_dist,jarow_dist,description_x_clean,description_y_clean,intersect_BOW,sym_diff_BOW,clusters,desc_sim,City_x,Neighborhood_x,State_x,Street_x,geocodeQuality_x,postalCode_x
0,1,81,182,126,31,50,300000.0,648140,0.0,64.686946,30.815924,4112648,1,1,81,243,194,132,60,300000.0,648140,0.0,64.686946,30.815924,1,1,648070,648070,1.0,0.0,0.000000,1.0,0,1.000000,1.000000,156,237,70,77,0,1.000000,0,0,30,0,3,0
1,3,14,50,34,74,52,300000.0,639040,0.0,55.678037,37.256548,1991275,1,1,14,55,60,70,62,330000.0,639040,0.0,55.678037,37.256548,1,1,637680,637680,1.0,30000.0,0.000000,1.0,1,0.925926,0.955556,233,353,281,112,0,0.698324,57,0,26,43,4,39
2,4,84,52,204,130,47,3500.0,640650,0.0,56.239398,43.460458,1223296,0,1,84,56,48,307,56,3500.0,640650,0.0,56.239398,43.460458,6,6,640310,640310,1.0,0.0,0.000000,0.9,2,0.931746,0.959048,229,169,258,278,0,0.902371,20,0,28,1,2,105
3,7,84,49,98,0,49,13500.0,662210,0.0,55.777170,37.586194,1058851,1,1,84,295,162,337,59,13500.0,662210,0.0,56.135459,47.235484,6,6,662000,662000,0.0,0.0,601.490989,1.0,9,0.788961,0.788961,113,171,63,77,0,1.000000,90,0,25,17,2,25
4,8,39,134,75,117,79,500.0,624360,0.0,55.777170,37.586194,2161930,1,1,39,77,35,6,89,600.0,624360,0.0,55.777170,37.586194,7,7,624300,624300,0.0,100.0,0.000000,0.0,13,0.360317,0.360317,78,41,105,189,0,0.983680,90,0,25,17,2,25


In [ ]:
learning_rate': 0.05,
 u'max_depth': 6,
 u'min_child_weight': 4,
 u'n_estimators': 1000

In [ ]:
model = xgb.XGBClassifier(learning_rate=0.05, max_depth=6, min_child_weight=4, n_estimators=1000)

In [ ]:
y = train['isDuplicate']
x = train.drop('isDuplicate', axis=1)

In [ ]:
seed = 1
test_size = 0.77
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size, random_state = seed)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
gbm = model.fit(x_train,y_train)

In [ ]:
pred = gbm.predict(x_test)

In [ ]:
pred_1 = [round(value) for value in pred]

In [ ]:
accuracy_1 = accuracy_score(y_test, pred_1)

In [ ]:
print('XGBoost Accuracy: {}'.format(accuracy_1 * 100.0))

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
roc_auc_score(y_test, gbm.predict(x_test))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fpr, tpr, _ = roc_curve(y_test, gbm.predict(x_test))

In [ ]:
plt.figure(figsize=(8, 8))
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve')
plt.plot([0, 1], [0, 1], color='lightblue', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
print(gbm.feature_importances_)

In [ ]:
ax = xgb.plot_importance(gbm)
fig = ax.figure
fig.set_size_inches(15,8)

### Use Feature Importance for feature selction

In [ ]:
import numpy as np
from sklearn.feature_selection import SelectFromModel

In [ ]:
thresholds = np.sort(gbm.feature_importances_)

In [ ]:
len(thresholds)

In [ ]:
lst =[]
for i, thresh in enumerate(thresholds):
#     print('Threshold:{}, {}'.format(thresh, i))
    # gbm is prefitted in the above 
    selection = SelectFromModel(gbm, threshold=thresh, prefit=True)
    # Output should be a matrix to input into model, no need to call fit
#     print(x_train.shape)
    select_x_train = selection.transform(x_train)
    
    # Train model
    selection_model = xgb.XGBClassifier(learning_rate=0.05, max_depth=6, min_child_weight=4, n_estimators=1000)
    selection_model.fit(select_x_train, y_train)
    
    # Evaluate model
    select_x_test = selection.transform(x_test)
    y_pred = selection_model.predict(select_x_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test,predictions)
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_x_train.shape[1], accuracy*100.0))

    
    lst.append(accuracy_score(y_test,predictions))    

In [ ]:
lst = np.array(lst) 
feature = np.array(range(0,47))

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(lst, color='darkorange', label='Accuracy curve')
plt.xlim([0, 47])
plt.ylim([0.6, 0.90])
plt.grid(True)
plt.show()

After dropping  more than 15 features, accuracy score starts to drop at a more significant rate.

In [ ]:
# Zoomed in view
plt.figure(figsize=(8, 8))
plt.plot(lst, color='darkorange', label='Accuracy curve')
plt.xlim([0, 14])
plt.ylim([0.824, 0.828])
plt.grid(True)
plt.show()

Optimal number of features to drop: 5 

### GridSearchCV

In [ ]:
from sklearn.grid_search import GridSearchCV

In [ ]:
params = {'learning_rate': [0, 0.01, 0.05, 0.5]
         ,'max_depth': [0, 2, 4, 6]
         ,'min_child_weight':[4, 7, 11]
         ,'n_estimators': [50, 100, 1000, 3000]}

In [ ]:
xgboost_gridsearch = GridSearchCV(gbm, params, n_jobs=5, scoring='roc_auc')

In [ ]:
xgboost_gridsearch.fit(x_train, y_train)

In [ ]:
xgboost_gridsearch.best_score_

In [ ]:
xgboost_gridsearch.best_params_

{u'learning_rate': 0.05,
 u'max_depth': 6,
 u'min_child_weight': 4,
 u'n_estimators': 1000}